<a href="https://colab.research.google.com/github/Mykhailo2009/E-commerce-User-Activity-Analysis/blob/main/e_commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
sql_code = """
WITH account_metrics AS (
    -- Обчислюємо кількість створених акаунтів у розрізі категоріальних значень
    SELECT
        s.date AS date,
        sp.country,
        a.send_interval,
        a.is_verified,
        a.is_unsubscribed,
        COUNT(DISTINCT a.id) AS account_cnt,  -- Кількість створених акаунтів
        0 AS sent_msg,
        0 AS open_msg,
        0 AS visit_msg
    FROM `data-analytics-mate.DA.account` a
    JOIN `data-analytics-mate.DA.account_session` acs
        ON a.id = acs.account_id
    JOIN `data-analytics-mate.DA.session` s
        ON acs.ga_session_id = s.ga_session_id
    JOIN `data-analytics-mate.DA.session_params` sp
        ON acs.ga_session_id = sp.ga_session_id
    GROUP BY date, sp.country, a.send_interval, a.is_verified, a.is_unsubscribed
),


email_metrics AS (
    -- Обчислюємо кількість відправлених, відкритих та клікнутих листів
    SELECT
       DATE_ADD(s.date, INTERVAL es.sent_date DAY) AS sent_date,  -- Обчислюємо дату відправлення
        sp.country,
        a.send_interval,
        a.is_verified,
        a.is_unsubscribed,
        0 AS account_cnt,
        COUNT(DISTINCT es.id_message) AS sent_msg,  -- Кількість відправлених листів
        COUNT(DISTINCT eo.id_message) AS open_msg,  -- Кількість відкритих листів
        COUNT(DISTINCT ev.id_message) AS visit_msg  -- Кількість переходів по листах
    FROM `data-analytics-mate.DA.email_sent` es
    JOIN `data-analytics-mate.DA.account` a
        ON es.id_account = a.id
    JOIN `data-analytics-mate.DA.account_session` acs
        ON a.id = acs.account_id
    JOIN `data-analytics-mate.DA.session` s
        ON acs.ga_session_id = s.ga_session_id
    JOIN `data-analytics-mate.DA.session_params` sp
        ON acs.ga_session_id = sp.ga_session_id
    LEFT JOIN `data-analytics-mate.DA.email_open` eo
        ON es.id_message = eo.id_message
    LEFT JOIN `data-analytics-mate.DA.email_visit` ev
        ON es.id_message = ev.id_message
    GROUP BY sent_date, sp.country, a.send_interval, a.is_verified, a.is_unsubscribed
),


combined_metrics AS (
    -- Об'єднуємо дані акаунтів та емейлів
    SELECT * FROM account_metrics
    UNION ALL
    SELECT * FROM email_metrics
),


aggregated_metrics AS (
    -- Остаточна агрегація даних у відповідних розрізах
    SELECT
        date, country, send_interval, is_verified, is_unsubscribed,
        SUM(account_cnt) AS account_cnt,
        SUM(sent_msg) AS sent_msg,
        SUM(open_msg) AS open_msg,
        SUM(visit_msg) AS visit_msg,
        -- Загальна кількість акаунтів по країні
        SUM(SUM(account_cnt)) OVER (PARTITION BY country) AS total_country_account_cnt,
        -- Загальна кількість відправлених листів по країні
        SUM(SUM(sent_msg)) OVER (PARTITION BY country) AS total_country_sent_cnt
    FROM combined_metrics
    GROUP BY date, country, send_interval, is_verified, is_unsubscribed
),


ranked_metrics AS (
    -- Ранжуємо країни за кількістю акаунтів та відправлених листів
    SELECT *,
        dense_rank() OVER (PARTITION BY date ORDER BY total_country_account_cnt DESC) AS rank_total_country_account_cnt,
        dense_rank() OVER (PARTITION BY date ORDER BY total_country_sent_cnt DESC) AS rank_total_country_sent_cnt
    FROM aggregated_metrics
)


-- Вибираємо тільки ТОП-10 країн за кількістю акаунтів або відправлених листів
SELECT *
FROM ranked_metrics
WHERE rank_total_country_account_cnt <= 10 OR rank_total_country_sent_cnt <= 10;

""".strip()